# Satellite Imagery

In this section we turn to **raster** data, specifically, to satellite imagery.  

In [1]:
import ibis
from ibis import _
con = ibis.duckdb.connect()

redlines = (con
            .read_geo("/vsicurl/https://dsl.richmond.edu/panorama/redlining/static/mappinginequality.gpkg")
            .filter(_.city == "New Haven")
            .execute()
           )
box = redlines.total_bounds
box

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

array([-72.99321,  41.23109, -72.85227,  41.37502])

In [2]:
from pystac_client import Client
import odc.stac
import pystac_client
import rioxarray
import geopandas as gpd
from rasterstats import zonal_stats 
import dask
workers = dask.distributed.Client()


AttributeError: module 'dask' has no attribute 'distributed'

In [ ]:
items = (
  Client.
  open("https://earth-search.aws.element84.com/v1").
  search(
    collections = ['sentinel-2-l2a'],
    bbox = box,
    datetime = "2024-06-01/2024-08-01",
    query={"eo:cloud_cover": {"lt": 5}}).
  item_collection()
)

In [ ]:
items

In [ ]:
data = odc.stac.load(
    items,
    bands=["nir08", "red"],
    bbox=box,
    chunks = {}
)

In [ ]:
ndvi = (
    ((data.nir08 - data.red) / (data.red + data.nir08))
    #.resample("time")
    .median("time", keep_attrs=True)
    .compute()
)

ndvi = ndvi.where(ndvi < 1)




In [ ]:
ndvi

In [ ]:
import matplotlib as plt
cmap = plt.colormaps.get_cmap('viridis')  # viridis is the default colormap for imshow
cmap.set_bad(color='black')

ndvi.plot.imshow( cmap=cmap, add_colorbar=False, size=4)